# Data preparation for Geneformer

In this notebook, we demonstrate how to prepare dataset in `.dataset` format for inference tasks. We use the the pancreas dataset described in the [scIB Github](https://github.com/theislab/scib). The transformation procedures containing the 3 main steps:

* Step 1: Convert gene name to Ensembl ID
* Step 2: Convert data from Anndata to Loom
* Step 3: Convert data from Loom to dataset

Then, the resulting data format will be used in analysis with `Geneformer` from finetuning on cell-type annotation to gene classification and beyonds. 

In [1]:
import os
import scanpy as sc
import numpy as np
import loompy
from geneformer import TranscriptomeTokenizer
from datasets import load_from_disk 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Step 1: Convert gene name to Ensembl ID

The current `AnnData` object uses gene names to indicate features on the gene expression matrix. However, `Geneformer` required `Ensembl ID` to indicate genes. Hence, the listed below codes will load the `AnnData` object and add the corresponding `Ensembl ID` of each gene to the `AnnData` object.

In [2]:
# # Download dataset
# !wget --timeout=0 --no-check-certificate 'https://s3.us-west-2.amazonaws.com/cdn.bioturing.com/colab/data/geneformer_data.zip'
# !unzip -o geneformer_data.zip

In [3]:
# # Download gene annotation information from ebi
# !wget  --timeout=0 --no-check-certificate https://s3.us-west-2.amazonaws.com/cdn.bioturing.com/colab/data/gencode.v44.annotation.gtf.gz

# os.system(f"/usr/bin/yes y | gunzip gencode.v44.annotation.gtf.gz")

In [4]:
data_h5ad = sc.read_h5ad('data/pancreas_scib.h5ad')
data_h5ad

AnnData object with n_obs × n_vars = 16382 × 19093
    obs: 'tech', 'celltype', 'size_factors', 'batch'
    layers: 'counts'

In [5]:
# Create a dictionary to map gene names to Ensembl IDs
ensembl_mapping = {}
with open('gencode.v44.annotation.gtf', 'r') as gtf_file:
    for line in gtf_file:
        if line.startswith('#'):
            continue
        parts = line.strip().split('\t')
        if parts[2] == 'gene':
            attributes = parts[-1].split('; ')
            gene_name = None
            gene_id = None
            for attr in attributes:
                if attr.startswith('gene_name'):
                    gene_name = attr.split('"')[1]
                elif attr.startswith('gene_id'):
                    gene_id = attr.split('"')[1]
            if gene_name and gene_id:
                ensembl_mapping[gene_name] = gene_id

In [6]:
# Function to convert gene names to Ensembl IDs
def convert_gene_name_to_ensembl(gene_name):
    return ensembl_mapping.get(gene_name, gene_name)  # Return the Ensembl ID if found, or the original gene name if not

In [7]:
# Apply the conversion function to the gene names in your AnnData object
data_h5ad.var['ensembl_id'] = data_h5ad.var_names.map(convert_gene_name_to_ensembl)

In [8]:
for i in range(len(data_h5ad.var['ensembl_id'])):
    data_h5ad.var['ensembl_id'][i] = data_h5ad.var['ensembl_id'][i].split(".")[0]

In [9]:
data_h5ad.var['ensembl_id']

A1BG      ENSG00000121410
A1CF      ENSG00000148584
A2M       ENSG00000175899
A2ML1     ENSG00000166535
A4GALT    ENSG00000128274
               ...       
ZXDC      ENSG00000070476
ZYG11B    ENSG00000162378
ZYX       ENSG00000159840
ZZEF1     ENSG00000074755
ZZZ3      ENSG00000036549
Name: ensembl_id, Length: 19093, dtype: object

Now, the AnnData object will have a new column `'ensembl_id'` with the Ensembl IDs. We can access it like this:

In [10]:
ensembl_ids = data_h5ad.var['ensembl_id']

In [11]:
for i in range(len(data_h5ad.var.index)):
    data_h5ad.var.index.values[i] = ensembl_ids[i]

We want to add another column indicating the number of genes in each cell in our analysis.  

In [12]:
data_h5ad.obs["n_counts"] = data_h5ad.obs.iloc[:,1]

In [13]:
data_h5ad.obs.iloc[:,1]

D101_5         gamma
D101_43        gamma
D101_93        gamma
D102_4         gamma
D172444_23     gamma
               ...  
Sample_1594    gamma
Sample_1595    gamma
Sample_1597    gamma
Sample_1598    gamma
Sample_1600    gamma
Name: celltype, Length: 16382, dtype: category
Categories (14, object): ['acinar', 'activated_stellate', 'alpha', 'beta', ..., 'mast', 'quiescent_stellate', 'schwann', 't_cell']

In [14]:
num_features_per_cell = np.sum(data_h5ad.X > 0, axis=1)

# Create a new column in adata.obs to store the number of features per cell
data_h5ad.obs['n_counts'] = num_features_per_cell

In [15]:
data_h5ad

AnnData object with n_obs × n_vars = 16382 × 19093
    obs: 'tech', 'celltype', 'size_factors', 'batch', 'n_counts'
    var: 'ensembl_id'
    layers: 'counts'

## Step 2: Convert data from AnnData to Loom

By using `write_loom` function, we can convert data from `AnnData` to `Loom` format.

In [16]:
data_h5ad.write_loom("./pancreas_scib.loom")

In [17]:
with loompy.connect('./pancreas_scib.loom') as ds:
    # Add 'cell_type' and 'organ_major' to the loom file
    print(ds.shape)
    print(ds.ra.keys())
    print(ds.ca.keys())
    print(ds[:,2])
    print(len(ds[:,2]))

(19093, 16382)
['ensembl_id', 'var_names']
['batch', 'celltype', 'n_counts', 'obs_names', 'size_factors', 'tech']
[0.       3.311074 0.       ... 0.       0.       0.      ]
19093


In [18]:
import loompy

with loompy.connect('./pancreas_scib.loom') as ds:
    unique_celltypes = set(ds.ca["celltype"])
    print(f"Number of unique cell types: {len(unique_celltypes)}")
    print("Unique cell types:")
    print(sorted(unique_celltypes))

Number of unique cell types: 14
Unique cell types:
[np.str_('acinar'), np.str_('activated_stellate'), np.str_('alpha'), np.str_('beta'), np.str_('delta'), np.str_('ductal'), np.str_('endothelial'), np.str_('epsilon'), np.str_('gamma'), np.str_('macrophage'), np.str_('mast'), np.str_('quiescent_stellate'), np.str_('schwann'), np.str_('t_cell')]


## Step 3: Convert data from Loom to dataset

We will use `TranscriptomeTokenizer` to convert dataset from `.loom` file to `.dataset`. This step also tokenizes our input dataset.

In [19]:
tk = TranscriptomeTokenizer(
    {
        "batch" : "batch", 
        "celltype" : "celltype", 
        "n_counts" : "n_counts", 
        "obs_names" : "obs_names", 
        "size_factors" : "size_factors", 
        "tech" : "tech"
    }, 
    nproc=20, 
    model_input_size = 2048,
    special_token=False # The 30M model series require the special_token argument to be set to False 
) 

<cls> and <eos> are in gene_token_dict but special_token = False. Please note that for 95M model series, special_token should be True.


In [20]:
import loompy

loom_path = "pancreas_scib.loom"
with loompy.connect(loom_path) as ds:
    print("Row attrs:", list(ds.ra.keys()))
    print("Col attrs:", list(ds.ca.keys()))

Row attrs: ['ensembl_id', 'var_names']
Col attrs: ['batch', 'celltype', 'n_counts', 'obs_names', 'size_factors', 'tech']


In [21]:
# Output tokenized data
tk.tokenize_data(
    data_directory="./",
    output_directory="./data",
    output_prefix="pancreas_scib",
)

Tokenizing pancreas_scib.loom


  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [00:12<00:00,  2.50it/s]


pancreas_scib__dedup.loom has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


## Load and check the input `.dataset` after tokenization 

In [22]:
token_dataset = load_from_disk("./data/pancreas_scib.dataset/")
print(type(token_dataset))
print(token_dataset)

<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['input_ids', 'batch', 'celltype', 'n_counts', 'obs_names', 'size_factors', 'tech', 'length'],
    num_rows: 16382
})


In [23]:
print(type(token_dataset["celltype"]))
# print(token_dataset["celltype"])

unique_cell_types = set(token_dataset["celltype"])
print(unique_cell_types) # 14 cell types

<class 'list'>
{'macrophage', 'schwann', 'activated_stellate', 'epsilon', 'beta', 'endothelial', 'acinar', 't_cell', 'quiescent_stellate', 'ductal', 'gamma', 'mast', 'alpha', 'delta'}


In [24]:
# Filter leaving only specific cell types
cell_types_to_keep = ["t_cell", "macrophage", "endothelial", "acinar"]
filtered_dataset = token_dataset.filter(lambda f: f["celltype"] in cell_types_to_keep)

In [25]:
print(filtered_dataset)
print(type(filtered_dataset))

unique_filtered_cell_types = set(filtered_dataset["celltype"])
print(unique_filtered_cell_types)

Dataset({
    features: ['input_ids', 'batch', 'celltype', 'n_counts', 'obs_names', 'size_factors', 'tech', 'length'],
    num_rows: 2068
})
<class 'datasets.arrow_dataset.Dataset'>
{'endothelial', 'acinar', 't_cell', 'macrophage'}


In [26]:
# Save the filtered dataset to disk
filtered_dataset.save_to_disk("./data/pancreas_scib_filtered.dataset/")

In [27]:
# Load the filtered dataset and check the cell types
loaded_filtered_dataset = load_from_disk("./data/pancreas_scib_filtered.dataset/")
print(loaded_filtered_dataset)
print(loaded_filtered_dataset.shape)
print(loaded_filtered_dataset[0])

Dataset({
    features: ['input_ids', 'batch', 'celltype', 'n_counts', 'obs_names', 'size_factors', 'tech', 'length'],
    num_rows: 2068
})
(2068, 8)
{'input_ids': [12663, 8145, 17778, 17478, 267, 13489, 5434, 12099, 2792, 19080, 2811, 18278, 15690, 3717, 9429, 10076, 16015, 16431, 17807, 9934, 2920, 7514, 213, 8172, 13109, 7954, 1152, 10715, 10568, 3500, 15221, 19969, 10430, 4227, 8680, 10509, 1009, 14371, 10281, 6339, 13343, 3130, 15219, 4525, 8778, 8962, 9466, 1932, 14641, 8264, 13357, 2677, 3457, 7987, 10534, 666, 12010, 5847, 7722, 1522, 7886, 6046, 18810, 10600, 3255, 335, 12677, 570, 12133, 823, 4708, 16411, 13409, 5595, 5735, 15177, 12602, 16640, 7250, 6196, 11969, 14352, 12035, 15338, 6479, 8942, 5479, 16306, 3362, 2925, 6105, 6889, 12005, 13157, 10245, 12777, 15240, 3316, 9971, 2384, 3232, 8519, 16142, 3101, 2313, 13593, 2196, 6712, 11031, 9437, 339, 9999, 5379, 7558, 4981, 10409, 4868, 8933, 18192, 16193, 6108, 12882, 8833, 3536, 14156, 9989, 660, 10586, 6488, 8929, 12538, 

In [28]:
print(token_dataset)
print(token_dataset.shape)
print(token_dataset[0])

Dataset({
    features: ['input_ids', 'batch', 'celltype', 'n_counts', 'obs_names', 'size_factors', 'tech', 'length'],
    num_rows: 16382
})
(16382, 8)
{'input_ids': [11840, 19969, 14358, 18662, 14036, 17364, 16254, 15963, 7108, 6530, 125, 8604, 1619, 16343, 10739, 6672, 7680, 13498, 5047, 11983, 13576, 14579, 9014, 13747, 10275, 5766, 13175, 3639, 5866, 3449, 16186, 443, 7679, 12488, 1339, 2338, 5667, 4811, 2178, 17666, 1522, 4739, 13469, 10446, 12469, 9399, 14319, 11749, 15600, 3316, 14289, 16683, 4263, 16124, 15440, 981, 16167, 4313, 9687, 12035, 603, 14625, 14255, 4979, 9163, 13357, 9535, 12218, 726, 1731, 7250, 18159, 3321, 6339, 6706, 3673, 7047, 2491, 14678, 7666, 339, 8109, 12624, 14594, 9846, 15856, 8678, 10964, 12643, 3536, 4527, 4320, 3515, 17546, 296, 1819, 8295, 9654, 3569, 14993, 21, 6235, 2745, 4868, 8707, 18810, 18250, 5301, 1919, 10788, 18638, 3161, 2520, 16008, 400, 6236, 2715, 5006, 14056, 1864, 3783, 12932, 12815, 12632, 3234, 11564, 11495, 7580, 3139, 4664, 9926, 